In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

from sklearn import metrics

In [ ]:
# import data
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [ ]:
train_x = train_data.reshape(train_data.shape[0], -1).astype('float32')
test_x = test_data.reshape(test_data.shape[0], -1).astype('float32')

# LightGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgb_model = LGBMClassifier(objective='multiclass',path_smooth = 0.2)

In [ ]:
lgb_model.fit(train_x,train_labels,categorical_feature=[0,3])

C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(objective='multiclass', path_smooth=0.2)

In [ ]:
expected_y  = test_labels
predicted_y = lgb_model.predict(test_x)
lightGBM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1000
           1       1.00      0.97      0.98      1000
           2       0.80      0.83      0.81      1000
           3       0.89      0.90      0.90      1000
           4       0.80      0.83      0.82      1000
           5       0.99      0.97      0.98      1000
           6       0.72      0.66      0.69      1000
           7       0.95      0.97      0.96      1000
           8       0.98      0.97      0.98      1000
           9       0.97      0.96      0.96      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [ ]:
# dont run this part, i'm still tuning
err_list = []
for i in np.arange(0.0, 1.0, 0.1):
    model = LGBMClassifier(objective='multiclass',path_smooth = i)
    model.fit(X_train,y_train,categorical_feature=[0,3])
    predictions = model.predict(X_test)
    error = sum(predictions!=y_test)/len(y_test)
    err_list.append(error)
err_list

In [ ]:
# dont run this part, i'm still tuning
err_list = []
for i in np.arange(0.0, 1.0, 0.1):
    model = LGBMClassifier(objective='multiclass',path_smooth = i)
    model.fit(X_train,y_train,categorical_feature=[0,3])
    predictions = model.predict(X_test)
    error = sum(predictions!=y_test)/len(y_test)
    err_list.append(error)
err_list

# LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
lda = LDA(n_components=9)

In [ ]:
expected_y  = test_labels
predicted_y = lda_model.predict(test_x)
lda_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80      1000
           1       1.00      0.93      0.96      1000
           2       0.71      0.70      0.71      1000
           3       0.81      0.86      0.83      1000
           4       0.71      0.74      0.73      1000
           5       0.89      0.90      0.89      1000
           6       0.56      0.57      0.57      1000
           7       0.88      0.89      0.89      1000
           8       0.94      0.93      0.93      1000
           9       0.92      0.91      0.91      1000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



C:\Users\asus\anaconda3\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(


In [ ]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.817801,0.7810,0.798977,1000.0000
1,0.998929,0.9330,0.964840,1000.0000
2,0.713415,0.7020,0.707661,1000.0000
3,0.807547,0.8560,0.831068,1000.0000
4,0.710878,0.7450,0.727539,1000.0000
5,0.890765,0.8970,0.893871,1000.0000
6,0.561765,0.5730,0.567327,1000.0000
7,0.884273,0.8940,0.889110,1000.0000
8,0.936299,0.9260,0.931121,1000.0000
9,0.916331,0.9090,0.912651,1000.0000


# SVM

In [ ]:
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
# Use Ploynomial kernel 
svmpoly = svm.SVC(kernel='poly', degree=3, C=1).fit(train_x, train_labels)

In [ ]:
expected_y  = test_labels
predicted_y = svmpoly.predict(test_x)
SVM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81      1000
           1       0.99      0.95      0.97      1000
           2       0.80      0.78      0.79      1000
           3       0.88      0.86      0.87      1000
           4       0.83      0.76      0.79      1000
           5       0.84      0.96      0.89      1000
           6       0.66      0.69      0.68      1000
           7       0.94      0.91      0.92      1000
           8       0.97      0.94      0.95      1000
           9       0.96      0.94      0.95      1000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000



In [ ]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.790019,0.839,0.813773,1000.000
1,0.991693,0.955,0.973001,1000.000
2,0.804145,0.776,0.789822,1000.000
3,0.878695,0.862,0.870268,1000.000
4,0.825708,0.758,0.790407,1000.000
5,0.835366,0.959,0.892924,1000.000
6,0.662500,0.689,0.675490,1000.000
7,0.936410,0.913,0.924557,1000.000
8,0.966084,0.940,0.952864,1000.000
9,0.963077,0.939,0.950886,1000.000


In [ ]:
# to get the precision dictionary
out = {}
for i in range(0,10):
    out.update({i:round(report[str(i)]['precision'],2)})

In [ ]:
# create df of y_pred of test set for each model
y_pred_test = pd.concat([pd.DataFrame(lightGBM_pred_y, columns = ['LightGBM']),
           pd.DataFrame(lda_pred_y, columns = ['LDA']), 
           pd.DataFrame(SVM_pred_y, columns = ['SVM'])],axis=1)

In [ ]:
y_pred_test.to_csv('y_pred_test.csv')

### User defined function

In [ ]:
# Construct the reference table
ref1 = {0: 0.84, 1: 1.0, 2: 0.8, 3: 0.89, 4: 0.8, 5: 0.99, 6: 0.72, 7: 0.95, 8: 0.98, 9: 0.97}
ref2 = {0: 0.82, 1: 1.0, 2: 0.71, 3: 0.81, 4: 0.71, 5: 0.89, 6: 0.56, 7: 0.88, 8: 0.94, 9: 0.92}
ref3 = {0: 0.79, 1: 0.99, 2: 0.8, 3: 0.88, 4: 0.83, 5: 0.84, 6: 0.66, 7: 0.94, 8: 0.97, 9: 0.96}
ref_df = [ref1, ref2, ref3]

In [ ]:
ypred_org = y_pred_test
ypred = ypred_org.copy()

In [ ]:
# represent the y_test_pred with according precision
for i in range(df2.shape[1]):
   ypred.iloc[:,i] = ypred.iloc[:,i].replace(ref_df[i])

ypred.columns = [1,2,3]
ypred

In [ ]:
# For each sample, take the result from model with highest precision 
n=ypred.idxmax(axis='columns')
result=[]
for j in range(len(n)):
    m=n[j]-1
    result.append(ypred_org.iloc[j,m])

In [ ]:
# Calculate the precision for udf model
metrics.classification_report(test_labels, result, output_dict=True)